# XAS Workflow Task 1 Batch processing

This notebook contains a modified version of the first task of the XAS processing workflow. In this case, the  

The break up of the task consist of the following steps 

| Task                            | Input                                         | Output
| -------------                   |-------------                                  | -----  
| Create Athena project      |                                               | 
| 1. Import data                |Files: 4000 Data \[.dat\] files                      | 
| 2. Normalisation              |No need as all data has been normalised in DAWN |
| 3. Save Athena Project        |                                               |Files: 4000 Athena \[.prj\] filwa

For more details about larch, see https://xraypy.github.io/xraylarch/xafs/

## Column and group names.

Knowing the elements of the input data set it is possible to start manipulating and visualising the XAFS data. 

In this case energy and $\mu$ are provided, but the names of the columns do not match the names used by Larch. We rename the columns. The name of the groups is the same as the name of the file. To make things easier we can also adjust the file name.


In [ ]:
# Larch Libraries
# library to read ascii files
from larch.io import read_ascii
# libraries to handle data groups
from larch.utils import group2dict, dict2group
# library to normalise data
from larch.xafs import pre_edge
# import the larch.io libraries for managing athena files
from larch.io import create_athena, read_athena, extract_athenagroup

# File handling
from pathlib import Path

#plotting library
%matplotlib inline
import matplotlib.pyplot as plt

#reading all with the same extension files from a dir
def get_files_list(source_dir, f_pattern):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob(f_pattern)):
        i_counter += 1
        files_list.append(filepath)
    return files_list



# Rename columns 
def rename_cols(xafs_group):
    # energy
    engy = xafs_group.col1
    # mu
    mu_e = xafs_group.col2
    # get a dictionary from te group
    xafs_dict = group2dict(xafs_group)
    # add mu and energy to the dictionary
    xafs_dict['energy'] = engy
    xafs_dict['mu'] = mu_e
    xafs_group = dict2group(xafs_dict)
    return xafs_group

# show plot of normalised data
def plot_normalised(xafs_group):
        plt.plot(xafs_group.energy, xafs_group.pre_edge, 'g', label='pre-edge') # plot pre-edge in green
        plt.plot(xafs_group.energy, xafs_group.post_edge, 'r', label='post-edge')# plot post-edge in green
        plt.plot(xafs_group.energy, xafs_group.mu, 'b', label=xafs_group.filename) # plot mu in blue
        plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
        plt.xlabel('Energy (eV)') # label y graph
        plt.ylabel('x$\mu$(E)') # label y axis
        plt.title("pre-edge and post_edge fitting to $\mu$")
        plt.legend() # show legend

# variables that can be changed to process different datasets
data_path = "..\\nexusdata\\rh4co_ox_53\\37123_Rh_4_CO_Oxidation_45_7_ascii\\"
file_pattern = "*.dat"
f_prefix = "rh4co"
top_count = 5000
i_count = 0
show_graph = False # False to prevent showing graphs


# create the path for storing results
base_path = Path("./" , f_prefix)
Path(base_path).mkdir(parents=True, exist_ok=True)

source_path = Path(data_path)
files_list = get_files_list(source_path, file_pattern)
xas_data = {}
for a_file in files_list:
    file_name = a_file.name
    
    print ("Processing:", file_name)
    print ("Path:", a_file)
    f_suffix = str(i_count).zfill(6) 
    p_name = f_prefix+f_suffix
    print ("project name:", p_name)
    p_path = Path(base_path , p_name + ".prj")
    print ("project path:", p_path)
    xas_data = read_ascii(a_file)
    # using vars(fe_xas) we see that the object has the following properties: 
    # path, filename, header, data, attrs, energy, xmu, i0
    # print(vars(xas_data))

    # rename columns and group
    xas_data = rename_cols(xas_data)
    # the group is the same as the file name
    xas_data.filename = p_name
    
    # calculate pre-edge and post edge and add them to group
    # using defaults
    pre_edge(energy=xas_data.energy, mu=xas_data.mu , group=xas_data)
    # Show graph if allowed
    if show_graph:
        plot_normalised(xas_data)
    
    xas_project = create_athena(p_path)
    xas_project.add_group(xas_data)
    xas_project.save()
    
    
    i_count +=1
    if i_count == top_count:
        break
       


Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00000.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00000.dat
project name: rh4co000000
project path: rh4co\rh4co000000.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00001.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00001.dat
project name: rh4co000001
project path: rh4co\rh4co000001.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00002.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00002.dat
project name: rh4co000002
project path: rh4co\rh4co000002.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00003.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00003.dat
project name: rh4co000003
project path: rh4co\rh4co000003.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00035.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00035.dat
project name: rh4co000035
project path: rh4co\rh4co000035.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00036.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00036.dat
project name: rh4co000036
project path: rh4co\rh4co000036.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00037.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00037.dat
project name: rh4co000037
project path: rh4co\rh4co000037.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00038.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00038.dat
project name: rh4co000038
project path: rh4co\rh4co000038.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00069.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00069.dat
project name: rh4co000069
project path: rh4co\rh4co000069.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00070.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00070.dat
project name: rh4co000070
project path: rh4co\rh4co000070.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00071.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00071.dat
project name: rh4co000071
project path: rh4co\rh4co000071.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00072.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00072.dat
project name: rh4co000072
project path: rh4co\rh4co000072.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00104.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00104.dat
project name: rh4co000104
project path: rh4co\rh4co000104.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00105.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00105.dat
project name: rh4co000105
project path: rh4co\rh4co000105.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00106.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00106.dat
project name: rh4co000106
project path: rh4co\rh4co000106.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00107.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00107.dat
project name: rh4co000107
project path: rh4co\rh4co000107.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00140.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00140.dat
project name: rh4co000140
project path: rh4co\rh4co000140.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00141.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00141.dat
project name: rh4co000141
project path: rh4co\rh4co000141.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00142.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00142.dat
project name: rh4co000142
project path: rh4co\rh4co000142.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00143.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00143.dat
project name: rh4co000143
project path: rh4co\rh4co000143.prj
Processing: 37123_Rh_4_CO_Oxidat

...

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01546.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01546.dat
project name: rh4co001546
project path: rh4co\rh4co001546.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01547.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01547.dat
project name: rh4co001547
project path: rh4co\rh4co001547.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01548.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01548.dat
project name: rh4co001548
project path: rh4co\rh4co001548.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01549.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01549.dat
project name: rh4co001549
project path: rh4co\rh4co001549.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01582.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01582.dat
project name: rh4co001582
project path: rh4co\rh4co001582.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01583.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01583.dat
project name: rh4co001583
project path: rh4co\rh4co001583.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01584.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01584.dat
project name: rh4co001584
project path: rh4co\rh4co001584.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01585.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01585.dat
project name: rh4co001585
project path: rh4co\rh4co001585.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01617.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01617.dat
project name: rh4co001617
project path: rh4co\rh4co001617.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01618.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01618.dat
project name: rh4co001618
project path: rh4co\rh4co001618.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01619.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01619.dat
project name: rh4co001619
project path: rh4co\rh4co001619.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01620.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01620.dat
project name: rh4co001620
project path: rh4co\rh4co001620.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01653.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01653.dat
project name: rh4co001653
project path: rh4co\rh4co001653.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01654.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01654.dat
project name: rh4co001654
project path: rh4co\rh4co001654.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01655.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01655.dat
project name: rh4co001655
project path: rh4co\rh4co001655.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01656.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01656.dat
project name: rh4co001656
project path: rh4co\rh4co001656.prj
Processing: 37123_Rh_4_CO_Oxidat

Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01689.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01689.dat
project name: rh4co001689
project path: rh4co\rh4co001689.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01690.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01690.dat
project name: rh4co001690
project path: rh4co\rh4co001690.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01691.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01691.dat
project name: rh4co001691
project path: rh4co\rh4co001691.prj
Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01692.dat
Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_01692.dat
project name: rh4co001692
project path: rh4co\rh4co001692.prj
Processing: 37123_Rh_4_CO_Oxidat